In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils._testing import ignore_warnings
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from numpy import asarray
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Promo Preprocessing

In [ ]:
promo_data_path = 'Promos_data.xlsx'
promo_data = pd.read_excel(promo_data_path)

# Convert 'Date' column to datetime format
promo_data['Date'] = pd.to_datetime(promo_data['Date'])

# Modify the time processing function to ensure it does not call strip on NaT objects, for promo,promo1 and so on
def round_to_last_hour(time_str):
    if pd.isna(time_str):
        return None
    # Remove any spaces from the time string and ensure AM or PM is uppercase
    time_str = time_str.strip().upper().replace('.', '')
    # Try to parse the time including minutes, if it fails try without minutes
    try:
        # Parse the time with minutes
        time_obj = pd.to_datetime(time_str, format='%I:%M%p', errors='coerce')
        # If parsing fails, try to parse without minutes
        if pd.isna(time_obj):
            time_obj = pd.to_datetime(time_str, format='%I%p', errors='coerce')
        # If it still fails, return None
        if pd.isna(time_obj):
            return None
        # Adjust to the nearest previous whole hour
        if time_obj.minute > 0 or time_obj.second > 0 or time_obj.microsecond > 0:
            time_obj = time_obj.replace(minute=0, second=0, microsecond=0)
            time_obj -= pd.Timedelta(hours=1)
        return time_obj.strftime('%I:%M%p')
    except Exception as e:
        # If any exception occurs, return None
        return None

# Iterate over all Timeframe columns and create new columns to store adjusted times
for i in range(1, 7):  # Includes all 6 promo campaigns
    start_time_col = f'Promo {i} Start Time'
    end_time_col = f'Promo {i} End Time'
    # Split Timeframe into start and end times
    promo_data[[start_time_col, end_time_col]] = promo_data[f'Promo {i} Timeframe'].str.split('-', expand=True)
    # Apply the function to adjust to the nearest previous whole hour
    promo_data[start_time_col] = promo_data[start_time_col].apply(round_to_last_hour)
    promo_data[end_time_col] = promo_data[end_time_col].apply(round_to_last_hour)

# Display the first few rows of adjusted data to verify
promo_data.head()


,Year,Month,Date,Day,Promo 1,Promo 1 Timeframe,Type1,Cost1,Rev1,Promo 2,...,Promo 2 Start Time,Promo 2 End Time,Promo 3 Start Time,Promo 3 End Time,Promo 4 Start Time,Promo 4 End Time,Promo 5 Start Time,Promo 5 End Time,Promo 6 Start Time,Promo 6 End Time
0,2012.0,Jan,2012-01-01,Sun,NaN,NaN,NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
1,2012.0,Jan,2012-01-02,Mon,NaN,NaN,NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
2,2012.0,Jan,2012-01-03,Tue,Winning Winter Match Game,10am-10pm,SwipeAndWin,2.0,2.0,NaN,...,None,None,None,None,None,None,None,None,None,None
3,2012.0,Jan,2012-01-04,Wed,NaN,NaN,NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
4,2012.0,Jan,2012-01-05,Thu,$25K Super Swipe,10am-10pm,SwipeAndWin,2.0,2.0,NaN,...,None,None,None,None,None,None,None,None,None,None


In [ ]:
promo_data.to_csv("promo.csv")

In [ ]:
excel_data_path = 'test.xlsx'
excel_data = pd.read_excel(excel_data_path))

promo_data['Date'] = pd.to_datetime(promo_data['Date'])
excel_data['Date'] = pd.to_datetime(excel_data['Date'])
# Defining the function to mark promo hours
def mark_promo_hours(df, promo_start, promo_end, promo_name, promo_value):
    # Create a mask for the hours when the promo is active
    mask = (df['Date'] >= promo_start) & (df['Date'] <= promo_end)
    # Mark these hours with the promo name
    df.loc[mask, promo_name] = promo_value
    return df

def mark_cost_revenue(df, promo_start, promo_end, cost, revenue, type1):
    # Create a mask for the hours when the cost and revenue should be marked
    mask = (df['Date'] >= promo_start) & (df['Date'] <= promo_end)
    # Update the 'Cost' and 'Revenue' for the matching hours
    df.loc[mask, 'Cost1'] += cost
    df.loc[mask, 'Rev1'] += revenue
    df.loc[mask, 'Type1'] = type1
    return df

# Initialize the 'Cost' and 'Revenue' columns to zero
excel_data['Cost1'] = 0.0
excel_data['Rev1'] = 0.0
excel_data['Type1'] = "0"

# Iterate over the promo_data to apply the cost and revenue
for index, row in promo_data.iterrows():
    # Check if the promo has a valid start and end time
    if pd.notnull(row['Promo 1 Start Time']) and pd.notnull(row['Promo 1 End Time']):
        promo_start_1 = pd.Timestamp(row['Date']).replace(hour=int(row['Promo 1 Start Time'].split(':')[0]))
        promo_end1 = pd.Timestamp(row['Date']).replace(hour=int(row['Promo 1 End Time'].split(':')[0]))
        if 'PM' in row['Promo 1 End Time'] and '12' not in row['Promo 1 End Time']:
            promo_end1 += pd.Timedelta(hours=12)  # adjust for PM times not including noon
        if 'PM' in row['Promo 1 Start Time'] and '12' not in row['Promo 1 Start Time']:
            promo_start_1 += pd.Timedelta(hours=12)  # adjust for PM times not including noon
        # Convert start and end times to datetime.time for comparison
        # Get the cost and revenue values from the row
        promo_cost =row['Cost1']
        promo_revenue =row['Rev1']
        promo_type = row['Type1']
        # Call the function to mark cost and revenue
        excel_data = mark_cost_revenue(excel_data, promo_start_1, promo_end1, promo_cost, promo_revenue, promo_type)

# Check the result after marking cost and revenue
excel_data.head()

# Process each promo in the original dataset
for i in range(1, 7):
    promo_name = f'Promo {i}'
    excel_data[promo_name] = 0  # initialize the promo columns with 0
    for _, row in promo_data.iterrows():
        # For each promo, check if there is a valid start and end time
        start_time_col = f'Promo {i} Start Time'
        end_time_col = f'Promo {i} End Time'
        if pd.notnull(row[start_time_col]) and pd.notnull(row[end_time_col]):
            # Parse the start and end times and align with the Excel file's datetime
            promo_start = pd.Timestamp(row['Date']).replace(hour=int(row[start_time_col].split(':')[0]))
            promo_end = pd.Timestamp(row['Date']).replace(hour=int(row[end_time_col].split(':')[0]))
            if 'PM' in row[end_time_col] and '12' not in row[end_time_col]:
                promo_end += pd.Timedelta(hours=12)  # adjust for PM times not including noon
            if 'PM' in row[start_time_col] and '12' not in row[start_time_col]:
                promo_start += pd.Timedelta(hours=12)  # adjust for PM times not including noon
            promo_value = row[promo_name] if promo_name in row else ""
            excel_data = mark_promo_hours(excel_data, promo_start, promo_end, promo_name,promo_value)



In [ ]:
excel_data.to_csv("result.csv")

In [ ]:
excel_data.columns#[['Promo 2']].value_counts()

Index(['Date', 'hour', 'weekday', 'month', 'year', 'Cost1', 'Rev1', 'Type1',
       'Promo 1', 'Promo 2', 'Promo 3', 'Promo 4', 'Promo 5', 'Promo 6'],
      dtype='object')

# Create Train Input File(Do not use this section if you already have a model to run)

## Merge with other files 2023 test

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import Training data
train_data=pd.read_csv("Final.csv")
train_data['Date']=pd.to_datetime(train_data['Date'])
train_data

<ipython-input-123-95d34622755f>:2: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data=pd.read_csv("/content/drive/Shareddrives/ENGIN-2023MDP_Motorcity_Project_Files/Final Deliverable/Final_11_28.csv")


,Date,CoinIn,Games,monthly_gas_price,slot,table,BKAP,BSAP,FAP,HAP,...,4week_ma_slot,cur_month_max_table,Promo2,Promo3,umcsent,holiday,category,hour,2week_bf_slot,2week_bf_table
0,2012-05-22 06:00:00,189313.7584,162462.3639,3.989,83.0,41.0,0,0,0,0,...,159.00,39.0,0,0,79.3,0,0,6,144.0,39.0
1,2012-05-22 07:00:00,185065.3526,161358.7276,3.989,66.0,46.0,0,0,0,0,...,131.50,48.0,0,0,79.3,0,0,7,113.0,32.0
2,2012-05-22 08:00:00,188805.0727,165433.5660,3.989,71.0,51.0,0,0,0,0,...,148.00,63.0,0,0,79.3,0,0,8,128.0,63.0
3,2012-05-22 09:00:00,200532.9186,174686.8790,3.989,137.0,85.0,0,0,0,0,...,206.50,99.0,0,0,79.3,0,0,9,198.0,95.0
4,2012-05-22 10:00:00,220248.8905,189118.6667,3.989,231.0,105.0,0,0,0,0,...,420.25,146.0,0,0,79.3,0,0,10,405.0,128.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93384,2023-01-16 06:00:00,33567.6700,20381.0000,3.429,66.0,18.0,0,0,0,0,...,68.25,26.0,0,0,64.9,Martin Luther King Jr. Day,State Holiday,6,97.0,26.0
93385,2023-01-16 07:00:00,58161.0700,32777.0000,3.429,51.0,13.0,0,0,0,0,...,59.75,27.0,0,0,64.9,Martin Luther King Jr. Day,State Holiday,7,85.0,18.0
93386,2023-01-16 08:00:00,75826.5300,45779.0000,3.429,57.0,12.0,0,0,0,0,...,82.50,23.0,0,0,64.9,Martin Luther King Jr. Day,State Holiday,8,118.0,16.0
93387,2023-01-16 09:00:00,130656.4800,85152.0000,3.429,117.0,14.0,0,0,0,0,...,146.50,26.0,0,0,64.9,Martin Luther King Jr. Day,State Holiday,9,148.0,23.0


In [ ]:
# Load test data or datasets you want to train
# load Date and other time related information
df=pd.read_excel("test.xlsx")
# Load Monthly gas price data
monthly_gas=pd.read_csv("APUS23B74716_gas_2023_Jan_til_now.csv")
# Load annotated holiday data
holiday= pd.read_excel('Holiday list With Edits .xlsx')
# Load Promotion data only
promo_away_games=excel_data.rename(columns={'Type1':"Type"})
# Load CoinIn, Games dataset
coinIn_games=pd.read_csv("HourlyCoinInGames2023.csv")
# Load Customer sentiment dataset
customer=pd.read_csv("/sca-table1-on-2023-Nov-27.csv")
# Load slot headcount dataset
slot_data=pd.read_csv("SlotHeadcounts2023.csv")
# Load table headcount dataset
table_data=pd.read_csv("TableHeadcounts2023.csv")
# Load sports away games dataset
sports_data=pd.read_excel("2023_sports_away_games.xlsx")

In [ ]:
# Convert all dataset's Date into python datetime datatype
df['Date']=pd.to_datetime(df['Date'])
promo_away_games['Date']=pd.to_datetime(promo_away_games['Date'])# promo
monthly_gas['DATE']=pd.to_datetime(monthly_gas['DATE'])
monthly_gas.rename(columns={'APUS23B74716':'monthly_gas_price','DATE':'Date'},inplace=True)# Rename column name to fit ML model intput
monthly_gas["Date"]=pd.to_datetime(monthly_gas["Date"]).dt.strftime('%Y-%m')
holiday['Date'] = pd.to_datetime(holiday['Date'],format="%m/%d/%Y")
holiday=holiday[['Date','holiday','category']]
holiday=holiday.drop_duplicates(subset=['Date']) # Drop dupblicate holiday date
coinIn_games['Date']=pd.to_datetime(coinIn_games['Date'],format="%m/%d/%y %I:%M %p")
coinIn_games.drop("AccountingDate",axis=1,inplace=True)
sports_data['Date'] = pd.to_datetime(sports_data['Date'])
slot_data['Date']=pd.to_datetime(slot_data['Date'],format="%m/%d/%y %I:%M %p")
table_data['Date']=pd.to_datetime(table_data['Date'],format="%m/%d/%y %I:%M %p")

# Create "Date" column from Customer Sentiment data
customer["Year"]=customer["Year"].astype(str)
customer["Month"]=customer["Month"].astype(str)
customer['Date']=customer[['Year', 'Month']].agg('-'.join, axis=1)
customer['Date']=pd.to_datetime(customer['Date'],format="%Y-%m").dt.strftime('%Y-%m')
customer.drop(["Unnamed: 3","Month","Year"],axis=1,inplace=True)
customer.rename(columns={"Index":"umcsent"},inplace=True)

# Extract cost and revenue data from promo
cost_revenue=excel_data[['Date','Cost1','Rev1']].rename(columns={'Cost1':'Cost','Rev1':'Revenue'})

# Rename and extract headcount from slot and table headcount data
slot_data.rename(columns={'Total':'slot'},inplace=True)
slot_data=slot_data[['Date','slot']]
table_data.rename(columns={'HeadCount':'table'},inplace=True)
table_data=table_data[['Date','table']]

# Select Exact Test data period if test dataset cannot directly concat with training data
df=df[df['Date']>'2023-01-16 10:00:00']
df['Date']=pd.to_datetime(df['Date'])

<ipython-input-125-8bce791a0c2c>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date']=pd.to_datetime(df['Date'])


In [ ]:
# Merge all test data into a result dataframe based on the 'Date' column
df=df.merge(monthly_gas,how='left',left_on=pd.to_datetime(df["Date"]).dt.strftime('%Y-%m'),right_on='Date').drop(['Date','Date_y'],axis=1).rename(columns={'Date_x':'Date'})
df=df.merge(holiday,how='left',left_on=pd.to_datetime(df["Date"]).dt.strftime('%Y-%m-%d'),right_on=pd.to_datetime(holiday["Date"]).dt.strftime('%Y-%m-%d')).drop(['Date_y','key_0'],axis=1).rename(columns={'Date_x':'Date'})
df=df.merge(promo_away_games[['Date','Promo 1', 'Promo 2','Promo 3']],left_on='Date',right_on='Date')
df=df.rename({'Promo 1':'Promo', 'Promo 2':'Promo2','Promo 3':'Promo3'},axis=1)
df=df.merge(coinIn_games,left_on='Date',right_on='Date')
df = pd.merge(df, sports_data, on='Date', how='left')
df = df.merge(customer,left_on=pd.to_datetime(df["Date"]).dt.strftime('%Y-%m'),right_on='Date').drop(['Date','Date_y'],axis=1).rename(columns={'Date_x':'Date'})
df = df.merge(cost_revenue,on='Date')
# df['day']=df['Date'].dt.day
df=df.merge(slot_data,left_on='Date',right_on='Date')
df=df.merge(table_data,left_on='Date',right_on='Date')
df

,Date,hour,weekday,month,year,monthly_gas_price,holiday,category,Promo,Promo2,...,HAP,BKAR,BSAR,FAR,HAR,umcsent,Cost,Revenue,slot,table
0,2023-01-16 11:00:00,11,0,1,2023,4.309,Martin Luther King Jr. Day,State Holiday,Surprise Slot Bonus,0,...,0,0,0,0,0,64.9,2.0,2.0,445,24
1,2023-01-16 12:00:00,12,0,1,2023,4.309,Martin Luther King Jr. Day,State Holiday,Surprise Slot Bonus,0,...,0,0,0,0,1,64.9,2.0,2.0,560,40
2,2023-01-16 13:00:00,13,0,1,2023,4.309,Martin Luther King Jr. Day,State Holiday,Surprise Slot Bonus,0,...,0,0,0,0,1,64.9,2.0,2.0,677,46
3,2023-01-16 14:00:00,14,0,1,2023,4.309,Martin Luther King Jr. Day,State Holiday,Surprise Slot Bonus,0,...,0,0,0,0,1,64.9,2.0,2.0,696,54
4,2023-01-16 15:00:00,15,0,1,2023,4.309,Martin Luther King Jr. Day,State Holiday,Surprise Slot Bonus,0,...,0,0,0,0,1,64.9,2.0,2.0,746,68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6759,2023-10-25 06:00:00,6,2,10,2023,4.516,NaN,NaN,0,0,...,0,0,0,0,0,63.8,0.0,0.0,35,0
6760,2023-10-25 07:00:00,7,2,10,2023,4.516,NaN,NaN,0,0,...,0,0,0,0,0,63.8,0.0,0.0,15,0
6761,2023-10-25 08:00:00,8,2,10,2023,4.516,NaN,NaN,0,0,...,0,0,0,0,0,63.8,0.0,0.0,25,0
6762,2023-10-25 09:00:00,9,2,10,2023,4.516,NaN,NaN,0,0,...,0,0,0,0,0,63.8,0.0,0.0,47,0


In [ ]:

df=pd.concat([train_data.drop(['darryl_table','darryl_slot','4week_ma_table','4week_ma_slot','cur_month_max_table','2week_bf_slot','2week_bf_table'],axis=1), df],ignore_index=True)
df

,Date,CoinIn,Games,monthly_gas_price,slot,table,BKAP,BSAP,FAP,HAP,...,Revenue,month,year,weekday,Promo2,Promo3,umcsent,holiday,category,hour
0,2012-05-22 06:00:00,189313.7584,162462.3639,3.989,83.0,41.0,0,0,0,0,...,0.0,5,2012,1,0,0,79.3,0,0,6
1,2012-05-22 07:00:00,185065.3526,161358.7276,3.989,66.0,46.0,0,0,0,0,...,0.0,5,2012,1,0,0,79.3,0,0,7
2,2012-05-22 08:00:00,188805.0727,165433.5660,3.989,71.0,51.0,0,0,0,0,...,0.0,5,2012,1,0,0,79.3,0,0,8
3,2012-05-22 09:00:00,200532.9186,174686.8790,3.989,137.0,85.0,0,0,0,0,...,0.0,5,2012,1,0,0,79.3,0,0,9
4,2012-05-22 10:00:00,220248.8905,189118.6667,3.989,231.0,105.0,0,0,0,0,...,2.0,5,2012,1,0,0,79.3,0,0,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100148,2023-10-25 06:00:00,19561.3600,11903.0000,4.516,35.0,0.0,0,0,0,0,...,0.0,10,2023,2,0,0,63.8,NaN,NaN,6
100149,2023-10-25 07:00:00,16471.3600,11236.0000,4.516,15.0,0.0,0,0,0,0,...,0.0,10,2023,2,0,0,63.8,NaN,NaN,7
100150,2023-10-25 08:00:00,47211.1900,17269.0000,4.516,25.0,0.0,0,0,0,0,...,0.0,10,2023,2,0,0,63.8,NaN,NaN,8
100151,2023-10-25 09:00:00,65251.7600,28536.0000,4.516,47.0,0.0,0,0,0,0,...,0.0,10,2023,2,0,0,63.8,NaN,NaN,9


In [ ]:
df.columns

Index(['Date', 'CoinIn', 'Games', 'monthly_gas_price', 'slot', 'table', 'BKAP',
       'BSAP', 'FAP', 'HAP', 'BKAR', 'BSAR', 'FAR', 'HAR', 'Promo', 'Type',
       'Cost', 'Revenue', 'month', 'year', 'weekday', 'Promo2', 'Promo3',
       'umcsent', 'holiday', 'category', 'hour'],
      dtype='object')

## Moving Average & Shifting Automatic Transformer

In [ ]:
# Create Customer Transformer to transform input data
class lag_transformer(BaseEstimator, TransformerMixin):
    """
    Shift for CoinIn, Games, slot headcount as staff schedule needs to be created before certain period.
    """
    def __init__(self,lag=24*21):
        """
        Initialize normal shift column and moving average columns.

        Input argument:
        lag: int
        number of hours to shift forward
        """
        self.column_to_transfer=['CoinIn','Games']
        self.ma_column_1='slot'
        self.ma_column_2='table'
        self.lag=lag

    @ignore_warnings()
    def fit(self, X,y):
        return self

    def transform(self,X):
        # Shift the CoinIn and Games column. Create 4 week, 12 week moving average of headcount. Get 2 week before same hour headcount
        X_copy=X.copy()
        X_copy[self.column_to_transfer]=X_copy[self.column_to_transfer].shift(self.lag)
        X_copy['darryl_table']=X_copy.groupby([X_copy['Date'].dt.dayofweek,X_copy['Date'].dt.hour])[self.ma_column_2].rolling(12).mean().shift(2).reset_index(level=0,drop=True).reset_index(level=0,drop=True)#.isna().sum()
        X_copy['darryl_slot']=X_copy.groupby([X_copy['Date'].dt.dayofweek,X_copy['Date'].dt.hour])[self.ma_column_1].rolling(12).mean().shift(2).reset_index(level=0,drop=True).reset_index(level=0,drop=True)#.reset_index(drop=True)
        X_copy['4week_ma_table']=X_copy.groupby([X_copy['Date'].dt.dayofweek,X_copy['Date'].dt.hour])[self.ma_column_2].rolling(4).mean().shift(2).reset_index(level=0,drop=True).reset_index(level=0,drop=True)#.isna().sum()
        X_copy['4week_ma_slot']=X_copy.groupby([X_copy['Date'].dt.dayofweek,X_copy['Date'].dt.hour])[self.ma_column_1].rolling(4).mean().shift(2).reset_index(level=0,drop=True).reset_index(level=0,drop=True)#.reset_index(drop=True)
        X_copy['cur_month_max_table']=X_copy.groupby([X_copy['Date'].dt.dayofweek,X_copy['Date'].dt.hour])[self.ma_column_2].rolling(4).max().shift(2).reset_index(level=0,drop=True).reset_index(level=0,drop=True)
        X_copy[['2week_bf_slot','2week_bf_table']]=X_copy[['slot','table']].shift(self.lag)
        X_copy=X_copy.iloc[self.lag:]
        return X_copy
def drop_nans(X, y=None):
    df = pd.DataFrame(X)
    subset='darryl_table'#'darryl_table'
    df.dropna(subset=[subset], inplace=True)
    return df#.values
class missing_encoder(BaseEstimator, TransformerMixin):
    """
    Fill missing value.
    """
    def __init__(self):
        self.num_mean_columns=['CoinIn','Games','monthly_gas_price']# column to fill with entire column mean
        self.num_const_column=['Cost','Revenue','umcsent'] # columns to fill with 0
        self.cate_const_columns=['holiday','category','Type'] # columns to fill with "0"
        self.num_imputer_1= SimpleImputer(strategy='mean')
        self.num_imputer_2= SimpleImputer(strategy='constant',fill_value=0)
        self.categorical_imputer_1=SimpleImputer(strategy="constant",fill_value="0")
    @ignore_warnings()
    def fit(self,X,y):
        x_copy=X.copy()
        self.num_imputer_1.fit(x_copy[self.num_mean_columns])
        self.num_imputer_2.fit(x_copy[self.num_const_column])
        self.categorical_imputer_1.fit(x_copy[self.cate_const_columns])
        return self
    @ignore_warnings()
    def transform(self,X):
        x_copy=X.copy()
        x_copy[self.num_mean_columns]=self.num_imputer_1.transform(x_copy[self.num_mean_columns])
        x_copy[self.num_const_column]=self.num_imputer_2.transform(x_copy[self.num_const_column])
        x_copy[self.cate_const_columns]=self.categorical_imputer_1.transform(x_copy[self.cate_const_columns])
        return x_copy
class value_encoder(BaseEstimator, TransformerMixin):
  """
  Use ordinal Encoding to encode categorical data.
  """
  def __init__(self):
      self.encoder1= OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=5000)
      self.num_columns=['holiday','category','Promo','Promo2','Promo3','Type']#,'Type'
  @ignore_warnings()
  def fit(self,X,y):
      x_copy=X.copy()
      x_copy[['Promo3','Promo2','Promo']] = x_copy[['Promo3','Promo2','Promo']].replace(0, 'zero')
      self.encoder1.fit(x_copy[self.num_columns])
      return self
  @ignore_warnings()
  def transform(self,X):
      x_copy=X.copy()
      x_copy[['Promo3','Promo2','Promo']] = x_copy[['Promo3','Promo2','Promo']].replace(0, 'zero')
      x_copy[self.num_columns]=self.encoder1.transform(x_copy[self.num_columns])
      return x_copy

In [ ]:
pipe = Pipeline([
        ("missing_value_imputer",missing_encoder()),
        ("shifter", lag_transformer()),
        ("dropper", FunctionTransformer(drop_nans, validate=False)),# drop the nan resulted from calculating moving average
        ("ordinal_encoder", value_encoder()),
    ])

pipe.fit(df.drop('hour',axis=1),df['hour'])
df2=pipe.transform(df)
df2

,Date,CoinIn,Games,monthly_gas_price,slot,table,BKAP,BSAP,FAP,HAP,...,holiday,category,hour,darryl_table,darryl_slot,4week_ma_table,4week_ma_slot,cur_month_max_table,2week_bf_slot,2week_bf_table
2184,2012-08-21 06:00:00,152520.5113,141093.5752,4.186,138.0,38.0,0,0,0,0,...,0.0,0.0,6,43.500000,127.416667,40.50,133.50,57.0,109.0,39.0
2185,2012-08-21 07:00:00,148272.1054,139989.9390,4.186,125.0,37.0,0,0,0,0,...,0.0,0.0,7,40.166667,109.500000,40.50,122.00,55.0,107.0,38.0
2186,2012-08-21 08:00:00,152011.8255,144064.7773,4.186,130.0,42.0,0,0,0,0,...,0.0,0.0,8,51.916667,120.000000,50.75,134.25,58.0,116.0,44.0
2187,2012-08-21 09:00:00,163739.6715,153318.0903,4.186,166.0,52.0,0,0,0,0,...,0.0,0.0,9,79.666667,162.833333,73.50,165.00,85.0,147.0,63.0
2188,2012-08-21 10:00:00,183455.6433,167749.8780,4.186,262.0,100.0,0,0,0,0,...,0.0,0.0,10,106.333333,278.916667,97.75,255.00,100.0,230.0,96.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100148,2023-10-25 06:00:00,117514.2800,26063.0000,4.516,35.0,0.0,0,0,0,0,...,0.0,0.0,6,13.750000,47.000000,9.75,49.00,14.0,58.0,10.0
100149,2023-10-25 07:00:00,56114.5000,20624.0000,4.516,15.0,0.0,0,0,0,0,...,0.0,0.0,7,9.333333,43.000000,6.75,43.50,10.0,43.0,10.0
100150,2023-10-25 08:00:00,83377.0500,31202.0000,4.516,25.0,0.0,0,0,0,0,...,0.0,0.0,8,10.750000,50.666667,10.75,46.75,15.0,42.0,13.0
100151,2023-10-25 09:00:00,131224.7200,66700.0000,4.516,47.0,0.0,0,0,0,0,...,0.0,0.0,9,12.583333,89.083333,12.75,90.00,16.0,90.0,10.0


In [ ]:
df2.columns

Index(['Date', 'CoinIn', 'Games', 'monthly_gas_price', 'slot', 'table', 'BKAP',
       'BSAP', 'FAP', 'HAP', 'BKAR', 'BSAR', 'FAR', 'HAR', 'Promo', 'Type',
       'Cost', 'Revenue', 'month', 'year', 'weekday', 'Promo2', 'Promo3',
       'umcsent', 'holiday', 'category', 'hour', 'darryl_table', 'darryl_slot',
       '4week_ma_table', '4week_ma_slot', 'cur_month_max_table',
       '2week_bf_slot', '2week_bf_table'],
      dtype='object')

In [ ]:
df2.to_csv("master_2023.csv")

# Create Prediction input file

In [ ]:
# Import Past data
train_data=pd.read_csv("master_2023.csv")
train_data['Date']=pd.to_datetime(train_data['Date'])
train_data=train_data[train_data['Date']<'2023-10-24 00:00:00']
train_data

,Unnamed: 0,Date,CoinIn,Games,monthly_gas_price,slot,table,BKAP,BSAP,FAP,...,holiday,category,hour,darryl_table,darryl_slot,4week_ma_table,4week_ma_slot,cur_month_max_table,2week_bf_slot,2week_bf_table
0,2184,2012-08-21 06:00:00,152520.5113,141093.5752,4.186,138.0,38.0,0,0,0,...,0.0,0.0,6,43.500000,127.416667,40.50,133.50,57.0,109.0,39.0
1,2185,2012-08-21 07:00:00,148272.1054,139989.9390,4.186,125.0,37.0,0,0,0,...,0.0,0.0,7,40.166667,109.500000,40.50,122.00,55.0,107.0,38.0
2,2186,2012-08-21 08:00:00,152011.8255,144064.7773,4.186,130.0,42.0,0,0,0,...,0.0,0.0,8,51.916667,120.000000,50.75,134.25,58.0,116.0,44.0
3,2187,2012-08-21 09:00:00,163739.6715,153318.0903,4.186,166.0,52.0,0,0,0,...,0.0,0.0,9,79.666667,162.833333,73.50,165.00,85.0,147.0,63.0
4,2188,2012-08-21 10:00:00,183455.6433,167749.8780,4.186,262.0,100.0,0,0,0,...,0.0,0.0,10,106.333333,278.916667,97.75,255.00,100.0,230.0,96.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97929,100113,2023-10-23 19:00:00,521514.1800,253548.0000,4.516,305.0,58.0,0,0,0,...,0.0,0.0,19,77.750000,640.000000,78.00,568.75,88.0,434.0,74.0
97930,100114,2023-10-23 20:00:00,595885.1600,271795.0000,4.516,283.0,68.0,0,0,0,...,0.0,0.0,20,76.166667,631.666667,74.75,581.75,84.0,490.0,84.0
97931,100115,2023-10-23 21:00:00,577274.7000,257905.0000,4.516,282.0,69.0,0,0,0,...,0.0,0.0,21,76.916667,572.750000,79.75,533.25,89.0,471.0,72.0
97932,100116,2023-10-23 22:00:00,491789.5600,228219.0000,4.516,265.0,61.0,0,0,0,...,0.0,0.0,22,83.250000,514.916667,83.75,481.75,96.0,422.0,79.0


In [ ]:
# Load test data or datasets you want to predict
# load Date and other time related information
df=pd.read_excel("test.xlsx")
# Load Monthly gas price data
monthly_gas=pd.read_csv("APUS23B74716_gas_2023_Jan_til_now.csv")
# Load annotated holiday data
holiday= pd.read_excel('Holiday list With Edits .xlsx')
# Load Promotion data only
promo_away_games=excel_data.rename(columns={'Type1':"Type"})
# Load CoinIn, Games dataset
coinIn_games=pd.read_csv("HourlyCoinInGames2023.csv")
# Load Customer sentiment dataset
customer=pd.read_csv("sca-table1-on-2023-Nov-27.csv")
# Load sports away games dataset
sports_data=pd.read_excel("2023_sports_away_games.xlsx")    

In [ ]:
# Convert all dataset's Date into python datetime datatype
df['Date']=pd.to_datetime(df['Date'])
promo_away_games['Date']=pd.to_datetime(promo_away_games['Date'])# promo
monthly_gas['DATE']=pd.to_datetime(monthly_gas['DATE'])
monthly_gas.rename(columns={'APUS23B74716':'monthly_gas_price','DATE':'Date'},inplace=True)# Rename column name to fit ML model intput
monthly_gas["Date"]=pd.to_datetime(monthly_gas["Date"]).dt.strftime('%Y-%m')
holiday['Date'] = pd.to_datetime(holiday['Date'],format="%m/%d/%Y")
holiday=holiday[['Date','holiday','category']]
holiday=holiday.drop_duplicates(subset=['Date']) # Drop dupblicate holiday date
coinIn_games['Date']=pd.to_datetime(coinIn_games['Date'],format="%m/%d/%y %I:%M %p")
coinIn_games.drop("AccountingDate",axis=1,inplace=True)
sports_data['Date'] = pd.to_datetime(sports_data['Date'])

# Create "Date" column from Customer Sentiment data
customer["Year"]=customer["Year"].astype(str)
customer["Month"]=customer["Month"].astype(str)
customer['Date']=customer[['Year', 'Month']].agg('-'.join, axis=1)
customer['Date']=pd.to_datetime(customer['Date'],format="%Y-%m").dt.strftime('%Y-%m')
customer.drop(["Unnamed: 3","Month","Year"],axis=1,inplace=True)
customer.rename(columns={"Index":"umcsent"},inplace=True)

# Extract cost and revenue data from promo
cost_revenue=excel_data[['Date','Cost1','Rev1']].rename(columns={'Cost1':'Cost','Rev1':'Revenue'})

# Rename and extract headcount from slot and table headcount data
slot_data.rename(columns={'Total':'slot'},inplace=True)
slot_data=slot_data[['Date','slot']]
table_data.rename(columns={'HeadCount':'table'},inplace=True)
table_data=table_data[['Date','table']]

# Select Exact Test data period if test dataset cannot directly concat with training data, essentially Date bigger than last column of past dataset
last_date=train_data['Date'].iloc[-1]
df=df[df['Date']>train_data['Date'].iloc[-1]]
df['Date']=pd.to_datetime(df['Date'])
df

<ipython-input-133-b8a193c6c311>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date']=pd.to_datetime(df['Date'])


,Date,hour,weekday,month,year
7104,2023-10-24 00:00:00,0,1,10,2023
7105,2023-10-24 01:00:00,1,1,10,2023
7106,2023-10-24 02:00:00,2,1,10,2023
7107,2023-10-24 03:00:00,3,1,10,2023
7108,2023-10-24 04:00:00,4,1,10,2023
7109,2023-10-24 05:00:00,5,1,10,2023
7110,2023-10-24 06:00:00,6,1,10,2023
7111,2023-10-24 07:00:00,7,1,10,2023
7112,2023-10-24 08:00:00,8,1,10,2023
7113,2023-10-24 09:00:00,9,1,10,2023


In [ ]:
# Merge all test data into a result dataframe based on the 'Date' column
df=df.merge(monthly_gas,how='left',left_on=pd.to_datetime(df["Date"]).dt.strftime('%Y-%m'),right_on='Date').drop(['Date','Date_y'],axis=1).rename(columns={'Date_x':'Date'})
df=df.merge(holiday,how='left',left_on=pd.to_datetime(df["Date"]).dt.strftime('%Y-%m-%d'),right_on=pd.to_datetime(holiday["Date"]).dt.strftime('%Y-%m-%d')).drop(['Date_y','key_0'],axis=1).rename(columns={'Date_x':'Date'})
df=df.merge(promo_away_games[['Date','Promo 1', 'Promo 2','Promo 3']],left_on='Date',right_on='Date')
df=df.rename({'Promo 1':'Promo', 'Promo 2':'Promo2','Promo 3':'Promo3'},axis=1)
df=df.merge(coinIn_games,left_on='Date',right_on='Date')
df = pd.merge(df, sports_data, on='Date', how='left')
df = df.merge(customer,left_on=pd.to_datetime(df["Date"]).dt.strftime('%Y-%m'),right_on='Date').drop(['Date','Date_y'],axis=1).rename(columns={'Date_x':'Date'})
df = df.merge(cost_revenue,on='Date')
df

,Date,hour,weekday,month,year,monthly_gas_price,holiday,category,Promo,Promo2,...,BSAP,FAP,HAP,BKAR,BSAR,FAR,HAR,umcsent,Cost,Revenue
0,2023-10-24 00:00:00,0,1,10,2023,4.516,NaN,NaN,0,0,...,0,0,0,0,0,0,0,63.8,0.0,0.0
1,2023-10-24 01:00:00,1,1,10,2023,4.516,NaN,NaN,0,0,...,0,0,0,0,0,0,0,63.8,0.0,0.0
2,2023-10-24 02:00:00,2,1,10,2023,4.516,NaN,NaN,0,0,...,0,0,0,0,0,0,0,63.8,0.0,0.0
3,2023-10-24 03:00:00,3,1,10,2023,4.516,NaN,NaN,0,0,...,0,0,0,0,0,0,0,63.8,0.0,0.0
4,2023-10-24 04:00:00,4,1,10,2023,4.516,NaN,NaN,0,0,...,0,0,0,0,0,0,0,63.8,0.0,0.0
5,2023-10-24 05:00:00,5,1,10,2023,4.516,NaN,NaN,0,0,...,0,0,0,0,0,0,0,63.8,0.0,0.0
6,2023-10-24 06:00:00,6,1,10,2023,4.516,NaN,NaN,0,0,...,0,0,0,0,0,0,0,63.8,0.0,0.0
7,2023-10-24 07:00:00,7,1,10,2023,4.516,NaN,NaN,0,0,...,0,0,0,0,0,0,0,63.8,0.0,0.0
8,2023-10-24 08:00:00,8,1,10,2023,4.516,NaN,NaN,0,0,...,0,0,0,0,0,0,0,63.8,0.0,0.0
9,2023-10-24 09:00:00,9,1,10,2023,4.516,NaN,NaN,0,0,...,0,0,0,0,0,0,0,63.8,0.0,0.0


In [ ]:
df=pd.concat([train_data.drop(['darryl_table','darryl_slot','4week_ma_table','4week_ma_slot','cur_month_max_table','2week_bf_slot','2week_bf_table'],axis=1), df],axis=0,ignore_index=True)
df = df.drop(['Unnamed: 0'],axis=1)
df

,Date,CoinIn,Games,monthly_gas_price,slot,table,BKAP,BSAP,FAP,HAP,...,Revenue,month,year,weekday,Promo2,Promo3,umcsent,holiday,category,hour
0,2012-08-21 06:00:00,152520.5113,141093.5752,4.186,138.0,38.0,0,0,0,0,...,0.0,8,2012,1,19.0,13.0,74.3,0.0,0.0,6
1,2012-08-21 07:00:00,148272.1054,139989.9390,4.186,125.0,37.0,0,0,0,0,...,0.0,8,2012,1,19.0,13.0,74.3,0.0,0.0,7
2,2012-08-21 08:00:00,152011.8255,144064.7773,4.186,130.0,42.0,0,0,0,0,...,0.0,8,2012,1,19.0,13.0,74.3,0.0,0.0,8
3,2012-08-21 09:00:00,163739.6715,153318.0903,4.186,166.0,52.0,0,0,0,0,...,0.0,8,2012,1,19.0,13.0,74.3,0.0,0.0,9
4,2012-08-21 10:00:00,183455.6433,167749.8780,4.186,262.0,100.0,0,0,0,0,...,0.0,8,2012,1,19.0,13.0,74.3,0.0,0.0,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97977,2023-10-25 19:00:00,239973.6100,152654.0000,4.516,NaN,NaN,0,0,0,0,...,0.0,10,2023,2,0,0,63.8,NaN,NaN,19
97978,2023-10-25 20:00:00,264235.6300,146051.0000,4.516,NaN,NaN,0,0,0,0,...,0.0,10,2023,2,0,0,63.8,NaN,NaN,20
97979,2023-10-25 21:00:00,271190.1300,133662.0000,4.516,NaN,NaN,0,0,0,0,...,0.0,10,2023,2,0,0,63.8,NaN,NaN,21
97980,2023-10-25 22:00:00,258240.9600,119725.0000,4.516,NaN,NaN,0,0,0,0,...,0.0,10,2023,2,0,0,63.8,NaN,NaN,22


In [ ]:
df_transformed= pipe.transform(df)
df_transformed=df_transformed[df_transformed['Date']>last_date]
df_transformed

,Date,CoinIn,Games,monthly_gas_price,slot,table,BKAP,BSAP,FAP,HAP,...,holiday,category,hour,darryl_table,darryl_slot,4week_ma_table,4week_ma_slot,cur_month_max_table,2week_bf_slot,2week_bf_table
97934,2023-10-24 00:00:00,343062.89,166123.0,4.516,NaN,NaN,0,0,0,0,...,0.0,0.0,0,73.083333,319.166667,61.25,293.25,74.0,331.0,64.0
97935,2023-10-24 01:00:00,305187.72,115946.0,4.516,NaN,NaN,0,0,0,0,...,0.0,0.0,1,63.333333,234.000000,52.75,203.00,67.0,202.0,67.0
97936,2023-10-24 02:00:00,186378.91,81009.0,4.516,NaN,NaN,0,0,0,0,...,0.0,0.0,2,49.666667,173.333333,41.50,153.50,50.0,190.0,46.0
97937,2023-10-24 03:00:00,129085.60,64025.0,4.516,NaN,NaN,0,0,0,0,...,0.0,0.0,3,37.000000,123.750000,35.25,116.50,42.0,157.0,33.0
97938,2023-10-24 04:00:00,95475.99,41310.0,4.516,NaN,NaN,0,0,0,0,...,0.0,0.0,4,22.666667,78.333333,21.00,68.00,35.0,103.0,20.0
97939,2023-10-24 05:00:00,79820.98,35659.0,4.516,NaN,NaN,0,0,0,0,...,0.0,0.0,5,19.916667,58.000000,20.25,53.75,25.0,78.0,21.0
97940,2023-10-24 06:00:00,30396.87,16398.0,4.516,NaN,NaN,0,0,0,0,...,0.0,0.0,6,13.500000,45.500000,14.50,40.50,22.0,66.0,14.0
97941,2023-10-24 07:00:00,23989.30,18743.0,4.516,NaN,NaN,0,0,0,0,...,0.0,0.0,7,9.833333,38.666667,8.50,36.75,14.0,56.0,8.0
97942,2023-10-24 08:00:00,56342.27,35329.0,4.516,NaN,NaN,0,0,0,0,...,0.0,0.0,8,9.416667,49.833333,8.25,51.25,15.0,80.0,15.0
97943,2023-10-24 09:00:00,95704.55,71273.0,4.516,NaN,NaN,0,0,0,0,...,0.0,0.0,9,9.833333,95.083333,12.00,112.75,19.0,172.0,14.0


In [ ]:
# data processing
data=df_transformed
data['time'] = pd.to_datetime(data['Date'])

# Drop Certain Columns
data['Date'] = pd.to_datetime(data['Date'])
data['day'] = data['Date'].dt.day
data['month'] = data['Date'].dt.month
data['year'] = data['Date'].dt.year
data['hour'] = data['Date'].dt.hour
data['weekday'] = data['Date'].dt.weekday
data['hou'] = data['Date'].dt.hour


encoder = OrdinalEncoder()
data['holiday'] = encoder.fit_transform(data['holiday'].values.reshape(-1, 1))
data['category'] = encoder.fit_transform(data['category'].values.reshape(-1, 1))
data['Type'] = encoder.fit_transform(data['Type'].values.reshape(-1, 1))
data = data.drop(['Promo3'], axis=1)
data['holiday'] = np.where(data['holiday'] != 0, 1, data['holiday'])# 0 label from ordinal encoding seems have great impact to the final prediction performance
data['category'] = np.where(data['category'] != 0, 1, data['category'])
data['Type'] = np.where(data['Type'] != 0, 1, data['Type'])

data = data.drop(['Promo2'], axis=1)
data = data.drop(['Promo'], axis=1)
data = data.drop(['BSAP'], axis=1)
data = data.drop(['FAP'], axis=1)
data = data.drop(['HAP'], axis=1)
data = data.drop(['BKAP'], axis=1)
data = data.drop(['FAR'], axis=1)
data = data.drop(['BSAR'], axis=1)
data = data.drop(['CoinIn'], axis=1)
data = data.drop(['Games'], axis=1)
data = data.drop(['4week_ma_table'], axis=1)
data = data.drop(['cur_month_max_table'], axis=1)
data = data.drop(['2week_bf_table'], axis=1)
data = data.drop(['darryl_table'], axis=1)

data

,Date,monthly_gas_price,slot,table,BKAR,HAR,Type,Cost,Revenue,month,year,weekday,umcsent,holiday,category,hour,darryl_slot,4week_ma_slot,2week_bf_slot
97934,2023-10-24 00:00:00,4.516,NaN,NaN,0,0,0.0,0.0,0.0,10,2023,1,63.8,0.0,0.0,0,319.166667,293.25,331.0
97935,2023-10-24 01:00:00,4.516,NaN,NaN,0,0,0.0,0.0,0.0,10,2023,1,63.8,0.0,0.0,1,234.000000,203.00,202.0
97936,2023-10-24 02:00:00,4.516,NaN,NaN,0,0,0.0,0.0,0.0,10,2023,1,63.8,0.0,0.0,2,173.333333,153.50,190.0
97937,2023-10-24 03:00:00,4.516,NaN,NaN,0,0,0.0,0.0,0.0,10,2023,1,63.8,0.0,0.0,3,123.750000,116.50,157.0
97938,2023-10-24 04:00:00,4.516,NaN,NaN,0,0,0.0,0.0,0.0,10,2023,1,63.8,0.0,0.0,4,78.333333,68.00,103.0
97939,2023-10-24 05:00:00,4.516,NaN,NaN,0,0,0.0,0.0,0.0,10,2023,1,63.8,0.0,0.0,5,58.000000,53.75,78.0
97940,2023-10-24 06:00:00,4.516,NaN,NaN,0,0,0.0,0.0,0.0,10,2023,1,63.8,0.0,0.0,6,45.500000,40.50,66.0
97941,2023-10-24 07:00:00,4.516,NaN,NaN,0,0,0.0,0.0,0.0,10,2023,1,63.8,0.0,0.0,7,38.666667,36.75,56.0
97942,2023-10-24 08:00:00,4.516,NaN,NaN,0,0,0.0,0.0,0.0,10,2023,1,63.8,0.0,0.0,8,49.833333,51.25,80.0
97943,2023-10-24 09:00:00,4.516,NaN,NaN,0,0,0.0,0.0,0.0,10,2023,1,63.8,0.0,0.0,9,95.083333,112.75,172.0


In [ ]:
data = data.drop(['slot', 'Date', 'table'], axis = 1)
data

,monthly_gas_price,BKAR,HAR,Type,Cost,Revenue,month,year,weekday,umcsent,holiday,category,hour,darryl_slot,4week_ma_slot,2week_bf_slot
97934,4.516,0,0,0.0,0.0,0.0,10,2023,1,63.8,0.0,0.0,0,319.166667,293.25,331.0
97935,4.516,0,0,0.0,0.0,0.0,10,2023,1,63.8,0.0,0.0,1,234.000000,203.00,202.0
97936,4.516,0,0,0.0,0.0,0.0,10,2023,1,63.8,0.0,0.0,2,173.333333,153.50,190.0
97937,4.516,0,0,0.0,0.0,0.0,10,2023,1,63.8,0.0,0.0,3,123.750000,116.50,157.0
97938,4.516,0,0,0.0,0.0,0.0,10,2023,1,63.8,0.0,0.0,4,78.333333,68.00,103.0
97939,4.516,0,0,0.0,0.0,0.0,10,2023,1,63.8,0.0,0.0,5,58.000000,53.75,78.0
97940,4.516,0,0,0.0,0.0,0.0,10,2023,1,63.8,0.0,0.0,6,45.500000,40.50,66.0
97941,4.516,0,0,0.0,0.0,0.0,10,2023,1,63.8,0.0,0.0,7,38.666667,36.75,56.0
97942,4.516,0,0,0.0,0.0,0.0,10,2023,1,63.8,0.0,0.0,8,49.833333,51.25,80.0
97943,4.516,0,0,0.0,0.0,0.0,10,2023,1,63.8,0.0,0.0,9,95.083333,112.75,172.0


In [ ]:
# model running
import pickle
from sklearn.metrics import mean_absolute_percentage_error
with open('randomforest_model.pkl', 'rb') as file:
    rf_regressor_loaded = pickle.load(file)
y_pred = rf_regressor_loaded.predict(data)

In [ ]:
y_pred

array([297.68737702, 215.49236885, 168.21959996, 127.36227555,
        69.82962143,  52.77716561,  50.00761324,  52.57264134,
        50.84053544, 100.47852726, 187.33485076, 302.47075621,
       374.50448416, 389.37594835, 412.08230959, 418.86169128,
       430.32754754, 457.3182453 , 487.40220042, 508.34128245,
       494.43517599, 480.71761769, 475.03931292, 419.01540673,
       316.60885461, 234.51205455, 169.75952475, 113.77911168,
        73.66281553,  52.71106263,  50.17660628,  52.22479182,
        51.03431236,  89.44112101, 154.93908001, 247.66125334,
       333.71685844, 382.87332141, 424.40357962, 446.80896513,
       444.19436829, 495.3181394 , 518.43011788, 516.80769963,
       529.52482341, 514.60218801, 513.55984756, 427.00250114])